# Автоматическое резюмирование медицинских публикаций: извлечение результатов и выводов

Импорт библиотек

In [2]:
import torch
print(torch.__version__)

2.1.0+cu118


In [3]:
import numpy as np
print(np.__version__)

1.23.5


In [4]:
import accelerate
print(accelerate.__version__)

1.8.1


In [43]:
import os
import zipfile
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from datasets import Dataset
from rouge_score import rouge_scorer
import random

In [9]:
from transformers import LEDTokenizer, LEDForConditionalGeneration
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq

Датасет SUMPUBMED содержит ~33 000 научных медицинских статей, собранных с платформы PubMed / BioMed Central (BMC). Предназначен для задачи абстрактного резюмирования — генерации кратких, содержательных аннотаций на основе полного текста статьи.

In [8]:
!kaggle datasets download -d chandrasekhardcs/sumpubmed-dataset

sumpubmed-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [9]:
zip_path = "./sumpubmed-dataset.zip"
extract_to = "./sumpubmed_dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

In [17]:
df = pd.read_csv("./sumpubmed_dataset/SUMPUBMED.csv")
df.head()

,Unnamed: 0,line_text,filename_text,text,shorter_abstract,abstract
0,0,BACKGROUND\nthe skeleton is a multifunctional ...,700,BACKGROUND\nthe skeleton is a multifunctional ...,"during mouse development, obif is initially ob...",BACKGROUND\nwhile several cell types are known...
1,1,BACKGROUND\nthe exact interactions of the sing...,10862,BACKGROUND\nthe exact interactions of the sing...,elective analysis of the effects of this drug ...,BACKGROUND\nglucocorticoids have been proven t...
2,2,BACKGROUND\nultra-high-throughput sequencing i...,21467,BACKGROUND\nultra-high-throughput sequencing i...,the processing and statistical analysis of suc...,BACKGROUND\nsolexa/illumina short-read ultra-h...
3,3,BACKGROUND\nlignin is a phenolic heteropolymer...,26308,BACKGROUND\nlignin is a phenolic heteropolymer...,many cad/cad-like genes do not seem to be asso...,BACKGROUND\ncinnamyl alcohol dehydrogenase pr...
4,4,BACKGROUND\nidentification of genetic variants...,9879,BACKGROUND\nidentification of genetic variants...,the frequency of lof variants differed greatly...,"BACKGROUND\nover the last few years, continuou..."


Модель LED (Longformer-Encoder-Decoder), которая поддерживает обработку длинных статей (до 16к токенов)

In [7]:
model_name = "allenai/led-base-16384"
tokenizer = LEDTokenizer.from_pretrained(model_name)
model = LEDForConditionalGeneration.from_pretrained(model_name, gradient_checkpointing=True, use_safetensors=True).to("cuda")
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [10]:
special_tokens = ["<results>", "<conclusions>"]
tokenizer.add_special_tokens({"additional_special_tokens": special_tokens})
model.resize_token_embeddings(len(tokenizer))

Embedding(50267, 768, padding_idx=1)

Подготовка данных к обучению

In [18]:
df.drop(columns=["Unnamed: 0", "line_text", "filename_text", "shorter_abstract"], inplace=True)

In [19]:
def extract_results_conclusions(text):
    pattern = r"(BACKGROUND.*?)(RESULTS.*?)(CONCLUSIONS.*?)(?=\n\n|$)"
    matches = re.search(pattern, text, re.DOTALL)
    cleaned = "".join([matches.group(2), matches.group(3)])
    return cleaned

In [20]:
def remove_section_headers(text):
    cleaned = re.sub(r"\b(?:RESULTS|CONCLUSIONS)\b[:\s]*", "", text, flags=re.IGNORECASE)
    cleaned = re.sub(r"\s*\n\s*", " ", cleaned)
    cleaned = re.sub(r"\s{2,}", " ", cleaned).strip()
    return cleaned

In [21]:
def preprocess_abstract(abstract):
    processed_abstract = (
        "<results> " + re.search(r"RESULTS(.*?)CONCLUSIONS", abstract, re.DOTALL).group(1).strip() +
        " <conclusions> " + abstract.split("CONCLUSIONS")[1].strip()
    )
    return processed_abstract

In [22]:
df['text'] = df['text'].apply(extract_results_conclusions)
df['abstract'] = df['abstract'].apply(extract_results_conclusions)

df['text'] = df['text'].apply(remove_section_headers)
df['abstract'] = df['abstract'].apply(preprocess_abstract)

Оставляем тексты до 4096 токенов и абстракты до 256 токенов

In [23]:
def checker(processed_text, processed_abstract):
    if len(tokenizer.tokenize(processed_text)) <= 4096 and len(tokenizer.tokenize(processed_abstract)) <= 256:
        return True
    return False

In [24]:
def preprocess_and_filter(df):
    to_drop = []
    
    for i in range(len(df)):
        if not checker(df.loc[i, 'text'], df.loc[i, 'abstract']):
            to_drop.append(i)
            
    df = df.drop(to_drop)
    return df.reset_index(drop=True)

In [25]:
df = preprocess_and_filter(df)
print(len(df))

11807


In [26]:
def create_dataset(df):
    return Dataset.from_dict({
        "input_text": df['text'].tolist(),
        "target_text": df['abstract'].tolist()
    })

In [27]:
train_df, val_df = train_test_split(df, test_size=0.1)

In [28]:
train_dataset = create_dataset(train_df)
val_dataset = create_dataset(val_df)

In [29]:
def tokenize_function(examples):
    inputs = tokenizer(
        examples["input_text"],
        max_length=4096,
        truncation=True,
        padding="max_length"
    )
    outputs = tokenizer(
        examples["target_text"],
        max_length=256,
        truncation=True,
        padding="max_length"
    )
    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": outputs["input_ids"]
    }

In [30]:
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 1181/1181 [00:21<00:00, 53.81 examples/s]


Обучение модели

In [24]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./led_summarization",  
    eval_strategy="steps",          
    eval_steps=500,                     
    save_steps=500,                       
    logging_steps=100,                    
    learning_rate=3e-5,                     
    per_device_train_batch_size=2,        
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,         
    warmup_steps=500,                    
    num_train_epochs=3,                     
    predict_with_generate=True,           
    fp16=True,                             
    load_best_model_at_end=True,          
    metric_for_best_model="eval_loss",      
    greater_is_better=False,
    report_to="tensorboard",               
)

In [25]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    processing_class=tokenizer
)

In [26]:
trainer.train()

  3%|▎         | 100/3987 [10:45<6:57:45,  6.45s/it]

{'loss': 5.8725, 'grad_norm': 17.915178298950195, 'learning_rate': 5.82e-06, 'epoch': 0.08}


  5%|▌         | 200/3987 [21:30<6:47:02,  6.45s/it]

{'loss': 3.5465, 'grad_norm': 12.345606803894043, 'learning_rate': 1.1760000000000001e-05, 'epoch': 0.15}


  8%|▊         | 300/3987 [32:14<6:36:23,  6.45s/it]

...

 98%|█████████▊| 3900/3987 [7:26:42<09:20,  6.45s/it]

{'loss': 1.5717, 'grad_norm': 2.3142495155334473, 'learning_rate': 7.829079437912246e-07, 'epoch': 2.94}


100%|██████████| 3987/3987 [7:36:32<00:00,  6.87s/it]

{'train_runtime': 27392.8616, 'train_samples_per_second': 1.164, 'train_steps_per_second': 0.146, 'train_loss': 1.8613673301756097, 'epoch': 3.0}


TrainOutput(global_step=3987, training_loss=1.8613673301756097, metrics={'train_runtime': 27392.8616, 'train_samples_per_second': 1.164, 'train_steps_per_second': 0.146, 'total_flos': 8.607712972426445e+16, 'train_loss': 1.8613673301756097, 'epoch': 3.0})

In [27]:
trainer.save_model(training_args.output_dir)
tokenizer.save_pretrained(training_args.output_dir)

('./led_summarization/tokenizer_config.json',
 './led_summarization/special_tokens_map.json',
 './led_summarization/vocab.json',
 './led_summarization/merges.txt',
 './led_summarization/added_tokens.json')

# Inference на тестовой выборке

In [42]:
pretrained_tokenizer = LEDTokenizer.from_pretrained("./led_summarization")
pretrained_model = LEDForConditionalGeneration.from_pretrained("./led_summarization").to("cuda")

In [51]:
sample_indices = random.sample(range(len(tokenized_val)), 3)

In [52]:
def format_medical_summary(generated_text):
    clean_text = generated_text.replace('</s>', '').replace('<s>', '').strip()
    results_section = ''
    conclusions_section = ''
    
    if '<results>' in clean_text:
        results_part = clean_text.split('<results>')[1]
        results_section = results_part.split('<conclusions>')[0].strip()
    
    if '<conclusions>' in clean_text:
        conclusions_part = clean_text.split('<conclusions>')[1]
        conclusions_section = conclusions_part.split('<dig>')[0].strip()
    
    formatted_output = ""
    if results_section:
        results_section = results_section[0].upper() + results_section[1:]
        formatted_output += "RESULTS:\n" + results_section + "\n\n"
    
    if conclusions_section:
        conclusions_section = conclusions_section[0].upper() + conclusions_section[1:]
        formatted_output += "CONCLUSIONS:\n" + conclusions_section
    
    return formatted_output.strip()

In [54]:
for idx in sample_indices:
    sample = tokenized_val[idx]
    input_ids = torch.tensor(sample["input_ids"]).unsqueeze(0).to("cuda")
    
    outputs = pretrained_model.generate(
        input_ids,
        max_length=256,
        num_beams=4,
        no_repeat_ngram_size=4,      
        repetition_penalty=2.0,
        early_stopping=True
    )
    
    generated_summary = pretrained_tokenizer.decode(outputs[0], skip_special_tokens=False)
    
    print("\nGenerated Summary:")
    print(format_medical_summary(generated_summary))
    print("\n" + "="*50)


Generated Summary:
RESULTS:
We developed a milp approach to compute for a given large metabolic network one or more minimum subnetworks preserving biological requirements that can be specified by the user. compared to previous work  <cit> , our method guarantees minimality of the subnetwork regarding the number of active reactions while preserving all the given requirements. in case there exist several minimum solutions, we are able to enumerate all of them. this may give additional insight how the network is functioning and which reactions are really needed to satisfy the requirements. we applied our algorithms to several genome-scale metabolic networks and we always found all the maximum subnetworks in reasonable time.

CONCLUSIONS:
We developed an milp approach using indicator variables and some other features of cplex. we implemented our algorithms in matlab and we were able to find all the minimum subsnetworks in normal time.


Generated Summary:
RESULTS:
Here we present peakanal

# Оценка качества модели: метрика ROUGE

In [14]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [32]:
def calculate_rouge(predictions, references):
    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []
    
    for pred, ref in zip(predictions, references):
        scores = scorer.score(ref, pred)
        rouge1_scores.append(scores['rouge1'].fmeasure)
        rouge2_scores.append(scores['rouge2'].fmeasure)
        rougeL_scores.append(scores['rougeL'].fmeasure)
    
    return {
        'rouge1': np.mean(rouge1_scores),
        'rouge2': np.mean(rouge2_scores),
        'rougeL': np.mean(rougeL_scores)
    }

In [38]:
def generate_predictions(model, tokenizer, tokenized_dataset, device="cuda"):
    predictions = []
    references = []
    
    model.to(device)
    model.eval()
    
    for sample in tokenized_dataset:
        input_ids = torch.tensor(sample["input_ids"]).unsqueeze(0).to(device)
        
        with torch.no_grad():
            output = model.generate(
                input_ids,
                max_length=256,
                num_beams=4,
                early_stopping=True,
                length_penalty=1.2,
                no_repeat_ngram_size=3,
                repetition_penalty=1.5
            )
        
        pred = tokenizer.decode(output[0], skip_special_tokens=True)
        ref = tokenizer.decode(sample["labels"], skip_special_tokens=True)
        
        predictions.append(pred)
        references.append(ref)
    
    return predictions, references

In [40]:
predictions, references = generate_predictions(pretrained_model, pretrained_tokenizer, tokenized_val)

In [41]:
rouge_scores = calculate_rouge(predictions, references)
print(f"ROUGE-1: {rouge_scores['rouge1']:.4f}")
print(f"ROUGE-2: {rouge_scores['rouge2']:.4f}")
print(f"ROUGE-L: {rouge_scores['rougeL']:.4f}")

ROUGE-1: 0.4547
ROUGE-2: 0.1718
ROUGE-L: 0.2569
